In [1]:
from warnings import warn

import numpy
from scipy import constants
from scipy.optimize import least_squares, minimize, root_scalar

In [59]:
class array:
    def __init__(self, isc, voc, imp, vmp, t, ns, np):
        assert isc[0] > imp[0] and isc[1] > imp[1], "Isc must exceed Imp."
        assert voc[0] > vmp[0] and voc[1] > vmp[1], "Voc must exceed Vmp."
        assert t > -273.15, "Temperature must exceed absolute zero."
        assert ns > 0, "There must be atleast one cell per string."
        assert np > 0, "There must be atleast one string per array."
        
        self.isc = isc
        self.voc = voc
        self.imp = imp
        self.vmp = vmp
        self.t = t
        self.ns = numpy.ceil(ns)
        self.np = numpy.ceil(np)

    def params(self, t, g):
        assert t > -273.15, "Temperature must exceed absolute zero."
        assert g >= 0, "Intensity must be non-negative."

        # Compute the adjusted cell parameters.
        dt = t - self.t
        isc = (self.isc[0] + dt * self.isc[1]) * g
        voc = self.voc[0] + dt * self.voc[1]
        imp = (self.imp[0] + dt * self.imp[1]) * g
        vmp = self.vmp[0] + dt * self.vmp[1]

        return isc, voc, imp, vmp

    def cell(self, t, g):
        isc, voc, imp, vmp = self.params(t, g)
        pmp = imp * vmp
        q_kT = constants.e / (constants.k * (t + 273.15))

        def x2eqn(x):
            # Diode model is a logarithmic function, V=F(I).
            i0, rs, n = x  # Parameters to be solved for numerically.
            i0 = i0 * 1e-20  # Scale factor to assist solver.
            q_nkT = q_kT / n
            v = lambda i: numpy.log((isc - i) / i0 + 1) / q_nkT - i * rs
            return v
        
        def x2params(x):
            v = x2eqn(x)
            
            # Numerically invert the diode model to solve for Isc/Imp.
            risc = root_scalar(
                f=v,
                x0=isc,
                bracket=(isc * 0.98, isc),
            )
            rimp = minimize(
                fun=lambda ximp: 1 / (ximp * v(ximp)),
                x0=isc / 2,
                bounds=[(0, isc * 0.99)],
            )
            assert risc.converged and rimp.success

            # Return the cell parameters.
            xisc = risc.root
            xvoc = v(0)
            ximp = rimp.x[0]
            xvmp = v(ximp)
            return xisc, xvoc, ximp, xvmp
        
        def minfun(x):
            # The IV-curve is optimized againt Voc, Vmp, and Pmp.
            _, xvoc, ximp, xvmp = x2params(x)
            return voc - xvoc, vmp - xvmp, pmp - (ximp * xvmp)

        result = least_squares(
            fun=minfun,
            x0=(1, 0.1, 2.5),
            bounds=((1e-3, 0, 0.1), (1e3, 1, 10)),
            xtol=None)
        emsg = "Curve fit failed for cell(t={:0.1f}, g={:0.2f})."
        assert result.success and result.cost < 0.01, emsg.format(t, g)

        xisc, xvoc, ximp, xvmp = x2params(result.x)

        # Warn if the solution is a poor fit.
        threshold = 0.02
        wmsg = "Curve fit {:s} error exceeds {:0.0f}%. target={:0.3f} solution={:0.3f}"
        if numpy.abs(1 - xisc/isc) > threshold:
            warn(wmsg.format("Isc", threshold * 100, isc, xisc))
        if numpy.abs(1 - xvoc/voc) > threshold:
            warn(wmsg.format("Voc", threshold * 100, voc, xvoc))
        if numpy.abs(1 - ximp/imp) > threshold:
            warn(wmsg.format("Imp", threshold * 100, imp, ximp))
        if numpy.abs(1 - xvmp/vmp) > threshold:
            warn(wmsg.format("Vmp", threshold * 100, vmp, xvmp))

        return xisc, xvoc, ximp, xvmp

    def curve(self, t, g):
        pass

In [60]:
# Azur Space 3G30A triple-junction solar cells in a 24s12p configuration.
# Isc/Imp are specified in (A, A/C), Voc/Vmp are specified in (V, V/C).
# Temperature is specified in C. Intensity is unitless and scales Isc/Imp.
array = array(
    isc=(0.5196, 0.00036),  # short-circuit current, temp coefficient
    voc=(2.690, -0.0062),   # open-circuit voltage, temp coefficient
    imp=(0.5029, 0.00024),  # max-power current, temp coefficient
    vmp=(2.409, -0.0067),   # max-power voltage, temp coefficient
    t=28,   # temperature at which the above parameters are specified
    ns=24,  # number of series cells in a string
    np=12,  # number of parallel strings in an array
)

In [61]:
array.params(t=80, g=1)

(0.5383199999999999, 2.3676, 0.5153800000000001, 2.0606)

In [73]:
array.cell(t=-140, g=0.3)

C:\Users\Mark\AppData\Local\Temp\ipykernel_11792\4047024964.py:89: UserWarning: Curve fit Imp error exceeds 2%. target=0.139 solution=0.134
  warn(wmsg.format("Imp", threshold * 100, imp, ximp))


(0.137736, 3.789286646865921, 0.13448804012893717, 3.473764307272473)

In [4]:
curve = array.curve(t=80, g=1)